In [1]:
import numpy as np
import scipy.linalg as la
from numpy.core.umath_tests import inner1d
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from matplotlib import pyplot as plt
from more_itertools import unique_everseen
import numba
from numba import *
import time
import timeit
from numpy.testing import assert_almost_equal
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
import matplotlib.cm as cm
from sklearn import cluster, datasets
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool, Value, Array
import multiprocessing

In [2]:
def dist(x,y):   
    return np.sum((x-y)**2)

In [8]:
def affinity(s, var):
    n = np.shape(s)[0]
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, i] = 0
            A[i,j] = np.exp(-(la.norm(s[i] - s[j])**2) / (2*var))
    return A

def originalkmeans(y, k, max_iter=300):
    idx = np.random.choice(len(y), k, replace=False)
    idx_data = y[idx]
    for i in range(max_iter):
        dist = np.array([inner1d(y-c, y-c) for c in idx_data])
        clusters = np.argmin(dist, axis=0)
        idx_data = np.array([y[clusters==i].mean(axis=0) for i in range(k)])
    return (clusters, idx_data)


def findbestvar(s,vari=np.linspace(.1, 5, 100),k=2):                  #to find best variance
    n = np.shape(s)[0]
    dis=[]
    for j in range(len(vari)):
        var=vari[j]
        A=affinity(s,var)
        D = np.zeros((n, n))
        for i in range(n):
            D[i, i] = 1 / (A[i].sum())


        L = np.sqrt(D).dot(A).dot(np.sqrt(D))

        value, vector = la.eig(L)

        idx = np.argsort(value)[::-1]
        value = value[idx]
        vector = vector[:, idx]

        X = vector[:, 0:k]
        Y = X / np.sum(X, 1)[:, np.newaxis]

        clusters,data=originalkmeans(Y, k, max_iter=300)

        snew=pd.DataFrame(s)
        snew['cluster'] = clusters
        b=pd.DataFrame(data)
        cluster=list(unique_everseen(clusters))
        b['cluster']=cluster
        x=pd.merge(snew, b, on='cluster', how='outer')
        c=x.ix[:,0:snew.shape[1]-1] 
        c=np.array(c)
        d=x.ix[:,snew.shape[1]:x.shape[1]]
        d=np.array(d)
        distance=dist(c,d)

        dis.append(distance)
    
    bestvar=np.argmin(dis)
    return(vari[bestvar])


In [ ]:
def finalclustering(s,k,var):
    n = np.shape(s)[0]
    A=affinity(s,var)
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = 1 / (A[i].sum())
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = 1 / (A[i].sum())

    L = np.sqrt(D).dot(A).dot(np.sqrt(D))

    value, vector = la.eig(L)

    idx = np.argsort(value)[::-1]
    value = value[idx]
    vector = vector[:, idx]

    X = vector[:, 0:k]
    Y = X / np.sum(X, 1)[:, np.newaxis]

    clusters,data=originalkmeans(Y, k, max_iter=10)
    final=np.concatenate((s, clusters.reshape((len(clusters),1))), axis = 1)
    return(final,data,clusters)

In [ ]:
s = np.array([[2,1], [3,4], [5,4]]) 

vari= np.linspace(.01, 4, 1000)
k = 2
bestvariance=findbestvar(s)
bestvariance

In [ ]:
finalclustering(s,k,var=bestvariance)

x,y,z=finalclustering(s,k,var=bestvariance)
print(x)
print(y)
print(z)

In [10]:
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs


batch_size = 50
centers = [[5, 5], [10, 10]]
n_clusters = len(centers)
X, labels_true = make_blobs(n_samples=500, centers=centers, cluster_std=0.7)

s=X

In [ ]:
vari= np.linspace(0.01, 10, 1000)
k = 2
bestvariance=findbestvar(s,vari,k)
#final,centers,clusters=finalclustering(s,k,var=bestvariance)

In [16]:
%%time

count = multiprocessing.cpu_count()

parts = np.array_split(s, count)
with multiprocessing.Pool() as pool:
    v= pool.map(findbestvar, parts)


CPU times: user 20.2 ms, sys: 38.8 ms, total: 59.1 ms
Wall time: 39.1 s


In [20]:
mean=np.mean(v)

In [21]:
def finalclustering(s,k=2,var=mean):
    n = np.shape(s)[0]
    A=affinity(s,var)
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = 1 / (A[i].sum())
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = 1 / (A[i].sum())

    L = np.sqrt(D).dot(A).dot(np.sqrt(D))

    value, vector = la.eig(L)

    idx = np.argsort(value)[::-1]
    value = value[idx]
    vector = vector[:, idx]

    X = vector[:, 0:k]
    Y = X / np.sum(X, 1)[:, np.newaxis]

    clusters,data=originalkmeans(Y, k, max_iter=10)
    final=np.concatenate((s, clusters.reshape((len(clusters),1))), axis = 1)
    return(final,data,clusters)

In [22]:
%%time


count = multiprocessing.cpu_count()

parts = np.array_split(s, count)
with multiprocessing.Pool() as pool:
    y= pool.map(finalclustering, parts)


CPU times: user 6.52 ms, sys: 26.3 ms, total: 32.9 ms
Wall time: 336 ms
